In [1]:
import glob
import os
import json
import pandas as pd
from tqdm.auto import tqdm

import numpy as np
from CotModel import CotModel
from LincModel import LincModel
from NaiveModel import NaiveModel

/root/miniconda3/envs/linc/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
glob.glob(os.path.join("results/folio", '*.json')) + glob.glob(os.path.join("results/proofwriter", '*.json'))

['results/folio/cot - Llama-2-13b-chat-hf (2024-11-29_06-45-56).json',
 'results/folio/cot - Llama-2-7b-chat-hf (2024-11-29_03-23-47).json',
 'results/folio/cot - Llama-3.1-8B-Instruct (2024-11-28_23-55-42).json',
 'results/folio/cot - Llama-3.2-1B-Instruct (2024-11-28_22-34-15).json',
 'results/folio/cot - Llama-3.2-3B-Instruct (2024-11-28_23-11-19).json',
 'results/folio/linc - Llama-2-13b-chat-hf (2024-12-03_03-17-15).json',
 'results/folio/linc - Llama-2-7b-chat-hf (2024-12-02_19-31-08).json',
 'results/folio/linc - Llama-3.1-8B-Instruct (2024-12-02_16-31-56).json',
 'results/folio/linc - Llama-3.2-1B-Instruct (2024-12-02_16-34-24).json',
 'results/folio/linc - Llama-3.2-3B-Instruct (2024-12-02_16-37-29).json',
 'results/folio/naive - Llama-2-13b-chat-hf (2024-11-28_23-06-36).json',
 'results/folio/naive - Llama-2-7b-chat-hf (2024-11-28_22-07-31).json',
 'results/folio/naive - Llama-3.1-8B-Instruct (2024-11-28_21-55-17).json',
 'results/folio/naive - Llama-3.2-1B-Instruct (2024-11-

In [3]:
paths = glob.glob(os.path.join("results/folio", '*.json')) + glob.glob(os.path.join("results/proofwriter", '*.json'))

def parse_cot(row):
    answers = [CotModel.parse(response) for response in row.responses]
    accuracies = [1 if answer == row["label"] else 0 for answer in answers]
    mean = np.mean(accuracies)
    return answers, accuracies, mean

def parse_linc(row):
    answers = [LincModel.parse(response) for response in row.responses]
    accuracies = [1 if answer == row["label"] else 0 for answer in answers]
    mean = np.mean(accuracies)
    return answers, accuracies, mean

def parse_naive(row):
    answers = [NaiveModel.parse(response) for response in row.responses]
    accuracies = [1 if answer == row["label"] else 0 for answer in answers]
    mean = np.mean(accuracies)
    return answers, accuracies, mean

def get_dataset_path(file_name):
    if "folio" in file_name:
        return "data/folio.csv"
    elif "proofwriter" in file_name:
        return "data/proofwriter.csv"
    else:
        raise Exception()

results_df_dict = {}

for path in tqdm(paths):
    with open(path, "r") as file:
        results = json.load(file)
    data_path = get_dataset_path(path)
    results_df = pd.DataFrame(results)
    results_df["label"] = pd.read_csv(data_path)["label"]

    if "cot" in path:
        results_df[["answers", "accuracies", "mean"]] = results_df.apply(lambda row: pd.Series(parse_cot(row)), axis=1)
    elif "linc" in path:
        results_df[["answers", "accuracies", "mean"]] = results_df.apply(lambda row: pd.Series(parse_linc(row)), axis=1)
    elif "naive" in path:
        results_df[["answers", "accuracies", "mean"]] = results_df.apply(lambda row: pd.Series(parse_naive(row)), axis=1)
    results_df_dict[path] = results_df


100%|██████████| 30/30 [21:35<00:00, 43.20s/it]


In [4]:
import pickle

with open("data/results_df_dict.pkl", "wb") as file:
    pickle.dump(results_df_dict, file)

In [6]:
import re

def get_model_choice(file_name):
    if "linc" in file_name:
        return "LINC"
    elif "naive" in file_name:
        return "Naive"
    elif "cot" in file_name:
        return "COT"
    else:
        raise Exception()


def get_model_name(file_name):
    pattern = r"Llama\S*"
    model = re.findall(pattern, file_name)[0]
    conversions = {
        "Llama-2-13b-chat-hf": "Llama 2 13B Chat",
        "Llama-2-7b-chat-hf": "Llama 2 7B Chat",
        "Llama-3.1-8B-Instruct": "Llama 3.1 8B Instruct",
        "Llama-3.2-1B-Instruct": "Llama 3.2 1B Instruct",
        "Llama-3.2-3B-Instruct": "Llama 3.2 3B Instruct",
    }

    for key, value in conversions.items():
        if key in model:
            return model.replace(key, value)
    raise Exception()

def get_dataset_name(file_name):
    if "folio" in file_name:
        return "FOLIO"
    elif "proofwriter" in file_name:
        return "ProofWriter"
    else:
        raise Exception()

all_results = []
for path in tqdm(paths):
    results_df = results_df_dict[path]
    results = {
        "dataset_name": get_dataset_name(path),
        "model_choice": get_model_choice(path),
        "model_name": get_model_name(path),
        "mean": results_df["mean"].mean(),
    }
    all_results.append(results)

100%|██████████| 30/30 [00:00<00:00, 4514.54it/s]


In [ ]:
all_results_df = pd.DataFrame(all_results)

In [8]:
all_results_df.to_csv("results/results.csv")

In [ ]:
example = results_df_dict["linc - Llama-3.1-8B-Instruct (2024-12-02_18-56-55).json"]
example

,index,responses,label,answers,accuracies,mean
0,0,[[<EVALUATE>\nTEXT: Dave is big.\nFOL: Big(dav...,Uncertain,"[Uncertain, Uncertain, Uncertain, Uncertain, U...","[1, 1, 1, 1, 1]",1.0
1,1,[[<EVALUATE>\nTEXT: The cat is cold.\nFOL: Col...,True,"[Uncertain, Uncertain, Uncertain, Uncertain, U...","[0, 0, 0, 0, 0]",0.0
2,2,[[<EVALUATE>\nTEXT: The cat does not chase the...,Uncertain,"[Uncertain, Uncertain, Uncertain, Uncertain, U...","[1, 1, 1, 1, 1]",1.0
3,3,[[<EVALUATE>\nTEXT: Rough and cold that is wha...,Uncertain,"[Uncertain, Error, Uncertain, Uncertain, Uncer...","[1, 0, 1, 1, 1]",0.8
4,4,[[<EVALUATE>\nTEXT: The cow is big.\nFOL: Big(...,True,"[Uncertain, Uncertain, Uncertain, Uncertain, U...","[0, 0, 0, 0, 0]",0.0
...,...,...,...,...,...,...
177,177,[[<EVALUATE>\nTEXT: The bald eagle eats the sq...,False,"[Uncertain, Uncertain, Uncertain, Uncertain, U...","[0, 0, 0, 0, 0]",0.0
178,178,[[<EVALUATE>\nTEXT: That guy Alan sure is nice...,False,"[Uncertain, Uncertain, Uncertain, Uncertain, U...","[0, 0, 0, 0, 0]",0.0
179,179,"[[<EVALUATE>\nTEXT: Bob may be round, but he i...",False,"[Uncertain, Uncertain, Uncertain, Uncertain, U...","[0, 0, 0, 0, 0]",0.0
180,180,[[<EVALUATE>\nTEXT: The cat eats the rabbit.\n...,Uncertain,"[False, False, False, False, False]","[0, 0, 0, 0, 0]",0.0


In [ ]:
pd.read_csv("data/proofwriter.csv")

,id,premises,question_id,conclusion,label
0,AttNoneg-OWA-D3-953,Dave is big.\r\nDave is green.\r\nDave is red....,Q10,Erin is nice.,Uncertain
1,RelNoneg-OWA-D3-1034,The cat is cold.\r\nThe cat is rough.\r\nThe c...,Q16,The dog needs the rabbit.,Uncertain
2,RelNeg-OWA-D1-829,The cat does not chase the rabbit.\r\nThe cat ...,Q5,The mouse does not visit the cat.,Uncertain
3,AttNonegNatLang-OWA-220,Rough and cold that is what they say about Blu...,Q11,Fred is not nice.,Uncertain
4,RelNoneg-OWA-D1-91,The cow is big.\r\nThe cow is blue.\r\nThe cow...,Q5,The squirrel does not like the squirrel.,Uncertain
...,...,...,...,...,...
177,RelNeg-OWA-D3-1572,The bald eagle eats the squirrel.\r\nThe squir...,Q7,The bald eagle is nice.,True
178,AttNonegNatLang-OWA-44,That guy Alan sure is nice.\r\nThe young perso...,Q16,Gary is green.,Uncertain
179,AttNonegNatLang-OWA-402,"Bob may be round, but he is also kind.\r\nChar...",Q1,Dave is kind.,True
180,RelNeg-OWA-D3-1427,The cat eats the rabbit.\r\nThe rabbit eats th...,Q6,The tiger is not round.,False


In [ ]:
example["responses"]

[["<EVALUATE>\nTEXT: Bob may be round, but he is also kind.\nFOL: (Round(bob) & -Kind(bob)) | Kind(bob)\nTEXT: Charlie may be round, but he is also kind.\nFOL: (Round(charlie) & -Kind(charlie)) | Kind(charlie)\nTEXT: Dave may be round, but he is also kind.\nFOL: (Round(dave) & -Kind(dave)) | Kind(dave)\nTEXT: Eric is green and cold too.\nFOL: Green(eric) & Cold(eric)\nTEXT: Every time you meet someone kind and nice, they'll be green, too.\nFOL: all x. (Kind(x) & Nice(x) -> Green(x))\nTEXT: When green, young and round fits a person, you'll see that rough will also fit.\nFOL: all x. ((Green(x) & Young(x) & Round(x)) -> Rough(x))\nTEXT: Every single big person is a little green in some areas.\nFOL: all x. (Big(x) -> exists y. Green(y) & InSomeAreas(y, x))\nTEXT: When rough and kind can describe a person, then cold will describe them, too.\nFOL: all x. ((Rough(x) & Kind(x)) -> Cold(x))\nTEXT: A kind person will certainly be rough as well.\nFOL: all x. (Kind(x) -> Rough(x))\nTEXT: Nice peop